## **bronze-companies-balance-sheet**
<p>This notebook extracts and stores raw balance sheet data for publicly listed companies from Yahoo Finance. It includes key financial figures such as assets, liabilities, and shareholder equity, forming the foundational layer for financial health and stability analysis.</p>

#### **Importing necessary libraries**

In [1]:
import yfinance as yf
import pandas as pd
import json
import datetime

StatementMeta(, 4404acff-8201-4405-8865-a9356dbbbbe0, 5, Finished, Available, Finished)

#### **Variables**

In [2]:
file_name = "comp_bal_sheet_data"
layer_name = "bronze"

StatementMeta(, 4404acff-8201-4405-8865-a9356dbbbbe0, 6, Finished, Available, Finished)

#### **Step 1: Load the company codes (tickers) from the CSV**

In [3]:
codes = spark.read.csv('Files/master_files/company_code.csv', header=True)
short_codes = [
    row['short_code'].strip()
    for row in codes.select('short_code').distinct().collect()
    if row['short_code']
]

StatementMeta(, 4404acff-8201-4405-8865-a9356dbbbbe0, 7, Finished, Available, Finished)

#### **Step 2: Fetch Balance Sheet for each ticker**

In [4]:
output = []

for i in short_codes:
    try:
        tickers = yf.Ticker(i)
        bal = tickers.balance_sheet

        if not bal.empty:
            bal['Company'] = i

            bal.reset_index(inplace=True)
            bal.rename(columns = {'index' : 'Breakdown'}, inplace=True)
            final_bal = bal.melt(id_vars = ['Breakdown','Company'],var_name='Date', value_name='Value')

            output.append(final_bal)
    except Exception as e:
        print(f"An error occurred for {i}: {e}")


StatementMeta(, 4404acff-8201-4405-8865-a9356dbbbbe0, 8, Finished, Available, Finished)

#### **Step 3: Combine all DataFrames and save for Bronze layer**

In [5]:
if output:
    final_bal_sht_df = pd.concat(output, ignore_index=True) 
    
    spark_df = spark.createDataFrame(final_bal_sht_df)

    spark_df.write.mode("overwrite").partitionBy('Date').parquet(f"Files/{layer_name}/{file_name}")

    print(f"[SUCCESS] Stock Balance sheet data written to Lakehouse")
else:
    print("[WARNING] No data to write.")

StatementMeta(, 2cafe600-252a-4bf1-8e24-cf2e047d31cf, 9, Finished, Available, Finished)

[SUCCESS] Stock Balance sheet data written to Lakehouse
